# Introduction to LangChain

## What is LangChain?
LangChain is a framework that helps you combine **language models** with various tools, APIs, and logic to create dynamic workflows and applications. It allows you to build **structured workflows** (LangGraphs) or flexible, real-time decision-makers (Agents) using large language models like GPT.

You can think of LangChain as a **bridge/chain** that connects language models to different tools, databases, or functions, enabling the model to perform more complex tasks.

---

## Example
Let’s say you want to build a **flight booking assistant** using LangChain. You can use a **LangGraph** (component of LangChain) to structure the assistant's tasks like this:
1. Ask for the user’s destination.
2. Search for available flights using an API.
3. Show the user the available flights.
4. Confirm the booking.

## Analogy:
Imagine LangChain as a GPS navigation system for your language models:

If you follow a predefined route (LangGraph), it guides you step-by-step to your destination.
But if there’s a detour or you decide to explore along the way, you can use an Agent to decide the best route dynamically, based on what’s happening in real-time.

**Refer to the slides for the workflow!**

## SetUp

In [65]:
# Download necessary
%pip install FastAPI langserve sse_starlette wikipedia python-dotenv langchain-openai langchain-core langchain langchain-community google-search-results langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 2.7 MB/s eta 0:00:00


In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

In [3]:
load_dotenv('template.env')

True

In [4]:
# Call in the model
# 'gpt-4'
# 'gpt-3.5-turbo'

openai_api_key = os.getenv("OPENAI_API_KEY")
gpt = ChatOpenAI(
    model='gpt-4o',
    temperature=0.7
)

In [5]:
# Track out cost and understand how langchain works
tracing = os.getenv("LANGCHAIN_TRACING_V2")
langsmith = os.getenv("LANGCHAIN_API_KEY")

In [6]:
gpt.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_45c6de4934', 'finish_reason': 'stop', 'logprobs': None}, id='run-7d0db531-2504-4bb1-8f84-f7c560276bca-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## LangChain Example (1)
Let's go back to the slides

In [7]:
from langchain_core.prompts import(
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import ChatOpenAI

In [8]:
# Define the prompt template with placeholders for genre and number of recommendations

# "system", "You are a movie critic who recommends movies based on genre."
# "human", "Recommend {movie_count} movies in the {genre} genre."

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie critic who recommends movies based on genre."),
        ("human", "Recommend {movie_count} movies in the {genre} genre."),
    ]
)

In [9]:
# Generate our few shot examples from a famous movie critic that you respect their opinion
examples = [
    {
        "input": ("human", "Recommend 3 movies in the science fiction genre."),
        "output": ("system", "Here are 3 must-watch science fiction movies:\n"
                    "1. Blade Runner 2049 (2017) – A visually stunning sequel that explores themes of identity and artificial intelligence.\n"
                    "2. The Matrix (1999) – A groundbreaking film that blends philosophy, action, and technology in a dystopian future.\n"
                    "3. Interstellar (2014) – A space epic that delves into the emotional and scientific challenges of interstellar travel.")
    },
    {
        "input": ("human", "Recommend 2 movies in the horror genre."),
        "output": ("system", "Here are 2 terrifying horror movies:\n"
                    "1. Hereditary (2018) – A chilling psychological horror film that explores family secrets and supernatural elements.\n"
                    "2. The Conjuring (2013) – Based on true events, this film is a masterclass in building suspense and delivering scares.")
    },
    {
        "input": ("human", "Recommend 4 movies in the action genre."),
        "output": ("system", "Here are 4 adrenaline-pumping action movies:\n"
                    "1. Mad Max: Fury Road (2015) – A high-octane chase across a post-apocalyptic desert with stunning visuals and practical effects.\n"
                    "2. John Wick (2014) – A sleek and stylish film featuring expertly choreographed fight sequences.\n"
                    "3. Die Hard (1988) – The ultimate action movie with a thrilling story of a cop fighting terrorists in a high-rise building.\n"
                    "4. The Dark Knight (2008) – A superhero film that combines action with a deep exploration of morality and chaos.")
    },
    {
        "input": ("human", "Recommend 5 movies in the romantic comedy genre."),
        "output": ("system", "Here are 5 delightful romantic comedies:\n"
                    "1. Notting Hill (1999) – A charming story of an ordinary bookstore owner falling for a famous actress.\n"
                    "2. 10 Things I Hate About You (1999) – A modern retelling of Shakespeare’s *The Taming of the Shrew*, set in high school.\n"
                    "3. Crazy Rich Asians (2018) – A romantic comedy that blends cultural identity with lavish settings and heartwarming moments.\n"
                    "4. When Harry Met Sally (1989) – A classic film exploring whether men and women can ever just be friends.\n"
                    "5. The Proposal (2009) – A hilarious and heartwarming movie about a fake engagement that turns into real love.")
    }
]


In [10]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples=examples,
)

In [11]:
# The final prompt we will use to prompt things from the LLM
final_prompt = ChatPromptTemplate.from_messages(
    [
        prompt_template,
        few_shot_prompt,
    ]
)

In [12]:
# Check that everything is working well
print(final_prompt.invoke({"genre": "science fiction", "movie_count": 3}).messages)

[SystemMessage(content='You are a movie critic who recommends movies based on genre.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Recommend 3 movies in the science fiction genre.', additional_kwargs={}, response_metadata={}), HumanMessage(content="('human', 'Recommend 3 movies in the science fiction genre.')", additional_kwargs={}, response_metadata={}), AIMessage(content="('system', 'Here are 3 must-watch science fiction movies:\\n1. Blade Runner 2049 (2017) – A visually stunning sequel that explores themes of identity and artificial intelligence.\\n2. The Matrix (1999) – A groundbreaking film that blends philosophy, action, and technology in a dystopian future.\\n3. Interstellar (2014) – A space epic that delves into the emotional and scientific challenges of interstellar travel.')", additional_kwargs={}, response_metadata={}), HumanMessage(content="('human', 'Recommend 2 movies in the horror genre.')", additional_kwargs={}, response_metadata={}), AIMessage(co

In [13]:
# Create the combined chain using LangChain Expression Language (LCEL) each of this block is a runnable.
chain = final_prompt | gpt | StrOutputParser()

In [14]:
# Run the chain with parameters for genre and number of recommendations
result = chain.invoke({"genre": "sci-fi", "movie_count": 5})

# Output
print(result)

Here are 5 delightful romantic comedies:

1. **Notting Hill (1999)** – A charming story of an ordinary bookstore owner falling for a famous actress.
2. **10 Things I Hate About You (1999)** – A modern retelling of Shakespeare’s *The Taming of the Shrew*, set in high school.
3. **Crazy Rich Asians (2018)** – A romantic comedy that blends cultural identity with lavish settings and heartwarming moments.
4. **When Harry Met Sally (1989)** – A classic film exploring whether men and women can ever just be friends.
5. **The Proposal (2009)** – A hilarious and heartwarming movie about a fake engagement that turns into real love.


### You are probably wondering what is StrOutputParser() ??
Well, it's a great way to force our output to be formatted a certain way!  
There are many outputs you can form using OutputParsers in LangChain.

The `StrOutputParser()` is just one of many types of parsers. It converts the output from the language model into a simple string format, which is useful when you want to ensure the output is clean and easy to handle.

---

### Types of OutputParser you can use:
LangChain provides several types of output parsers that you can use depending on the format you need for the task. Here's a list of some common ones:

1. **StrOutputParser**: Parses the output as a plain string.
2. **JsonOutputParser**: Ensures that the output is formatted as valid JSON.
3. **RegexOutputParser**: Uses regular expressions to parse specific parts of the output.
4. **BooleanOutputParser**: Converts the output into a boolean (True/False) based on the content.
5. **ListOutputParser**: Formats the output as a list of items.
6. **CsvOutputParser**: Parses the output into CSV format.
7. **DictOutputParser**: Parses the output as a Python dictionary.

For more detailed information about **OutputParser** and how to use them, you can check out the official [LangChain OutputParser documentation](https://docs.langchain.com/docs/modules/chains/prompts/output_parsers).


In [15]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
import re

In [16]:
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @field_validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

In [17]:
parser = PydanticOutputParser(pydantic_object=Joke)

In [18]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [19]:
chain = prompt | gpt | parser

In [20]:
output = chain.invoke({"query": "Tell me a joke."})
print(output)

setup="Why don't scientists trust atoms?" punchline='Because they make up everything!'


## Intermediate Langchain

### Feedback provider

In [21]:
from langchain.schema.runnable import RunnableBranch, RunnableLambda

In [22]:
# Define prompt templates for different feedback types
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a thank you note for this positive feedback: {feedback}."
        ),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
          "Generate a response addressing this negative feedback: {feedback}."
        ),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

# Define the feedback classification template
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."
        ),
    ]
)

In [23]:
# Define the runnable branches for handling feedback
branches = RunnableBranch(
    (
        lambda x: "positive" in x,
        positive_feedback_template | gpt | StrOutputParser()  # Positive feedback chain
    ),
    (
        lambda x: "negative" in x,
        negative_feedback_template | gpt | StrOutputParser()  # Negative feedback chain
    ),
    (
        lambda x: "neutral" in x,
        neutral_feedback_template | gpt | StrOutputParser()  # Neutral feedback chain
    ),
    escalate_feedback_template | gpt | StrOutputParser()
)

uppercase_output = RunnableLambda(lambda x: x.upper())
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}\n{x}")

# Create the classification chain
classification_chain = classification_template | gpt | StrOutputParser()

# Combine classification and response generation into one chain
chain = classification_chain | branches | uppercase_output | count_words

# Run the chain with an example review

# Good review - "The product is excellent. I really enjoyed using it and found it very helpful."
# Bad review - "The product is terrible. It broke after just one use and the quality is very poor."
# Neutral review - "The product is okay. It works as expected but nothing exceptional."
# Default - "I'm not sure about the product yet. Can you tell me more about its features and benefits?"

review = "The product is terrible. It broke after just one use and the quality is very poor."
result = chain.invoke({"feedback": review})

# Output the result
print(result)


Word count: 71
THANK YOU FOR YOUR FEEDBACK. WE APPRECIATE YOU TAKING THE TIME TO SHARE YOUR THOUGHTS WITH US. WE'RE SORRY TO HEAR THAT YOUR EXPERIENCE HASN'T MET YOUR EXPECTATIONS. YOUR FEEDBACK IS INCREDIBLY VALUABLE, AS IT HELPS US IDENTIFY AREAS FOR IMPROVEMENT. PLEASE LET US KNOW IF THERE ARE SPECIFIC CONCERNS OR ISSUES WE CAN ADDRESS. WE'RE COMMITTED TO MAKING THINGS RIGHT AND ENSURING A POSITIVE EXPERIENCE FOR YOU IN THE FUTURE.


## Now that we have a good grasp of how LCEL works time to try Agents!

#### Let's go to the slides to understand better how agents work and better understand it's usecase

#### tools parameters, https://api.python.langchain.com/en/latest/agents/langchain.agents.load_tools.load_tools.html

In [24]:
from langchain.agents import load_tools, create_react_agent, initialize_agent, AgentType, AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool

In [25]:
tools = load_tools(["serpapi", "llm-math"], llm=gpt)
agent_executor = initialize_agent(
    tools, gpt, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)


<ipython-input-25-5a8afbcc2827>:2: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent_executor = initialize_agent(


In [26]:
# Play around with average and median
agent_executor.run(
    "What's the average price of a 1 bedroom condo \
        in New York City in 2022. Calculate a 20%% deposit\
            for it."
)

<ipython-input-26-06608f0eb17b>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run(




> Entering new AgentExecutor chain...
To answer this question, I need to find the average price of a 1-bedroom condo in New York City in 2022 and then calculate a 20% deposit based on that price.

Action: Search
Action Input: "average price of 1 bedroom condo in New York City 2022"
Observation: ['The average sale price for a condo ranges from $785,333 for a studio apartment to $10,899,279 for 4+ bedroom apartments. Meanwhile, the average ...', 'During the downturn period of 2017-2019, median price declined to $1.588 million. Median price in 2022 was $1.746 million, a new record ...', 'Ownership is still markedly more expensive in Manhattan compared to renting, since the average rent for a 1 bedroom was $4,466 during the month ...', 'The average apartment rent in Manhattan is $4939. Browse detailed statistics & rent trends, compare apartment sizes and rent prices by neighborhood.', 'The cost of studios starts from 650,000 $ (4,121,000 ¥); · The cost of a 1-bedroom apartment varies fro

'The average price of a 1-bedroom condo in New York City in 2022 is approximately $1,400,000. A 20% deposit for it would be $280,000.'

## LangChain Agent Deep Dive

#### Understanding tools a little better
#### Creating a chatbot using ReAct agents

In [27]:
import datetime
from wikipedia import summary
from langchain import hub

In [28]:
# Define Tools DATETIME tool
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""
    import datetime

    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")

In [29]:
def search_wikipedia(query):
    """Searches Wikipedia and returns the summary of the first result."""
    from wikipedia import summary

    try:
        # Limit to two sentences for brevity
        return summary(query, sentences=2)
    except:
        return "I couldn't find any information on that."

In [30]:
# Define the tools that the agent can use
tools = [
    Tool(
        name="Time",
        func=get_current_time,
        description="Useful for when you need to know the current time.", #context
    ),
    Tool(
        name="Wikipedia",
        func=search_wikipedia,
        description="Useful for when you need to know information about a topic.",
    ),
]

In [31]:
# Load the correct JSON Chat Prompt from the hub
prompt = hub.pull("hwchase17/structured-chat-agent")
print(prompt.input_variables)

['agent_scratchpad', 'input', 'tool_names', 'tools']


In [32]:
# Create a structured Chat Agent with Conversation Buffer Memory

# ConversationBufferMemory stores the conversation history, allowing the agent to maintain context across interactions

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

<ipython-input-32-cec9c3d3b677>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [33]:
# create_structured_chat_agent initializes a chat agent designed to interact using a structured prompt and tools
# It combines the language model (llm), tools, and prompt to create an interactive agent

agent = create_structured_chat_agent(
    llm=gpt,
    tools=tools,
    prompt=prompt
)

In [34]:
# AgentExecutor is responsible for managing the interaction between the user input, the agent, and the tools
# It also handles memory to ensure context is maintained throughout the conversation

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,  # Use the conversation memory to maintain context
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
)

In [35]:
# Initial system message to set the context for the chat
# SystemMessage is used to define a message from the system to the agent, setting initial instructions or context

initial_message = """
  You are an AI assistant that can provide helpful answers using available tools.\n
  If you are unable to answer, you can use the following tools: Time and Wikipedia.
"""

memory.chat_memory.add_message(SystemMessage(content=initial_message))

In [36]:
# Chat Loop to interact with the user
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    # Add the user's message to the conversation memory
    memory.chat_memory.add_message(HumanMessage(content=user_input))

    # Invoke the agent with the user input and the current chat history
    response = agent_executor.invoke({"input": user_input})
    print("Bot:", response["output"])

    # Add the agent's response to the conversation memory
    memory.chat_memory.add_message(AIMessage(content=response["output"]))

User: exit


### LangChain Tools Deep Dive

In [37]:
# Docs: https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/

from langchain_core.tools import StructuredTool, Tool
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [38]:
def greet_user(name: str) -> str:
    """Greets the user by name."""
    return f"Hello, {name}!"


def reverse_string(text: str) -> str:
    """Reverses the given string."""
    return text[::-1]


def concatenate_strings(a: str, b: str) -> str:
    """Concatenates two strings."""
    return a + b

In [39]:
# Pydantic model for tool arguments
class ConcatenateStringsArgs(BaseModel):
    a: str = Field(description="First string")
    b: str = Field(description="Second string")

In [40]:
# Create tools using the Tool and StructuredTool constructor approach
tools = [
    # Use Tool for simpler functions with a single input parameter.
    # This is straightforward and doesn't require an input schema.
    Tool(
        name="GreetUser",  # Name of the tool
        func=greet_user,  # Function to execute
        description="Greets the user by name.",  # Description of the tool
    ),
    # Use Tool for another simple function with a single input parameter.
    Tool(
        name="ReverseString",  # Name of the tool
        func=reverse_string,  # Function to execute
        description="Reverses the given string.",  # Description of the tool
    ),
    # Use StructuredTool for more complex functions that require multiple input parameters.
    # StructuredTool allows us to define an input schema using Pydantic, ensuring proper validation and description.
    StructuredTool.from_function(
        func=concatenate_strings,  # Function to execute
        name="ConcatenateStrings",  # Name of the tool
        description="Concatenates two strings.",  # Description of the tool
        args_schema=ConcatenateStringsArgs,  # Schema defining the tool's input arguments
    ),
]

In [41]:
# Pull the prompt template from the hub
prompt = hub.pull("hwchase17/openai-tools-agent")

In [42]:
# Create the ReAct agent using the create_tool_calling_agent function
agent = create_tool_calling_agent(
    llm=gpt,  # Language model to use
    tools=tools,  # List of tools available to the agent
    prompt=prompt,  # Prompt template to guide the agent's responses
)

In [43]:
# Create the ReAct agent using the create_tool_calling_agent function
agent = create_tool_calling_agent(
    llm=gpt,  # Language model to use
    tools=tools,  # List of tools available to the agent
    prompt=prompt,  # Prompt template to guide the agent's responses
)

In [44]:
# Create the agent executor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,  # The agent to execute
    tools=tools,  # List of tools available to the agent
    verbose=True,  # Enable verbose logging
    handle_parsing_errors=True,  # Handle parsing errors gracefully
)

In [45]:
# Test the agent with sample queries
response = agent_executor.invoke({"input": "Greet Alice"})
print("Response for 'Greet Alice':", response)

response = agent_executor.invoke({"input": "Reverse the string 'hello'"})
print("Response for 'Reverse the string hello':", response)

response = agent_executor.invoke({"input": "Concatenate 'hello' and 'world'"})
print("Response for 'Concatenate hello and world':", response)



> Entering new AgentExecutor chain...

Invoking: `GreetUser` with `Alice`


Hello, Alice!Hello, Alice!

> Finished chain.
Response for 'Greet Alice': {'input': 'Greet Alice', 'output': 'Hello, Alice!'}


> Entering new AgentExecutor chain...

Invoking: `ReverseString` with `hello`


ollehThe reverse of the string 'hello' is 'olleh'.

> Finished chain.
Response for 'Reverse the string hello': {'input': "Reverse the string 'hello'", 'output': "The reverse of the string 'hello' is 'olleh'."}


> Entering new AgentExecutor chain...

Invoking: `ConcatenateStrings` with `{'a': 'hello', 'b': 'world'}`


helloworldThe concatenation of "hello" and "world" is "helloworld".

> Finished chain.
Response for 'Concatenate hello and world': {'input': "Concatenate 'hello' and 'world'", 'output': 'The concatenation of "hello" and "world" is "helloworld".'}


# Langgraph

### LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence.

In [47]:
import os
from openai import OpenAI

messages=[
        {
            "role": "user",
            "content": "Get a list of 100 attractions in Singapore for tourists",
        },
    ]

response = gpt.invoke(messages)

In [48]:
print(response.content)

Certainly! Singapore is a vibrant city with a plethora of attractions for tourists. Here’s a list of 100 attractions you might consider exploring:

1. Marina Bay Sands
2. Gardens by the Bay
3. Sentosa Island
4. Universal Studios Singapore
5. Merlion Park
6. Singapore Flyer
7. Orchard Road
8. Clarke Quay
9. Singapore Botanic Gardens
10. Chinatown
11. Little India
12. Arab Street
13. Raffles Hotel
14. Singapore Zoo
15. Night Safari
16. Jurong Bird Park
17. River Safari
18. ArtScience Museum
19. National Museum of Singapore
20. Asian Civilisations Museum
21. Esplanade – Theatres on the Bay
22. Fort Canning Park
23. Haw Par Villa
24. East Coast Park
25. Pulau Ubin
26. Changi Beach Park
27. Bukit Timah Nature Reserve
28. Sungei Buloh Wetland Reserve
29. Kusu Island
30. St John’s Island
31. Lazarus Island
32. Southern Ridges
33. Henderson Waves
34. Mount Faber Park
35. MacRitchie Reservoir
36. Coney Island
37. Punggol Waterway Park
38. S.E.A. Aquarium
39. Adventure Cove Waterpark
40. Madame 

In [49]:
prompt = """\
    Please give more information about a travel destination.
    please include address, category, recommendated visit duration and description
    when you give the recommendation
"""

In [50]:
messages=[
    {
        "role": "system",
        "content": prompt,
    },
    {
        "role": "user",
        "content": "Get a list of 100 attractions in Singapore for tourists",
    },
]

response = gpt.invoke(messages)

In [51]:
response.content.strip()

"Certainly! Here's a detailed overview of one of Singapore's most iconic attractions:\n\n### Marina Bay Sands SkyPark\n\n**Address:** 10 Bayfront Avenue, Singapore 018956\n\n**Category:** Observation Deck/Entertainment Complex\n\n**Recommended Visit Duration:** 2-3 hours\n\n**Description:**  \nThe Marina Bay Sands SkyPark is a must-visit destination for anyone traveling to Singapore. Perched atop the iconic Marina Bay Sands Hotel, the SkyPark offers breathtaking panoramic views of the city skyline, the Marina Bay area, and beyond. At 57 stories high, visitors can marvel at the architectural wonders of Singapore from the observation deck, which spans the length of the hotel's three towers.\n\nThe SkyPark is not just an observation area; it features a stunning infinity pool (exclusive to hotel guests), beautifully landscaped gardens, and several upscale dining options. The SkyPark is also home to the CE LA VI rooftop bar, where you can enjoy a cocktail while watching the sun set over the

In [52]:
prompt = """\
    Please give more information about a travel destination.
    please include address, category, recommendated visit duration and description.
    recommended visit duration should be in minutes.
    when you give the recommendation.

    Please make sure the response is in valid JSON format.

    For example:
    {
        "attractions": [
            {
                "name": "Marina Bay Sands",
                "address": "10 Bayfront Ave, Singapore 018956",
                "category": "Hotel",
                "recommended_visit_duration": "120",
                "description": "Marina Bay Sands is an integrated resort fronting Marina Bay in Singapore. At its opening in 2010, it was billed as the world's most expensive standalone casino property at S$8 billion, including the land cost."
            }
        ]
    }
"""

In [53]:
messages=[
    {
        "role": "system",
        "content": prompt,
    },
    {
        "role": "user",
        "content": "Get a list of 100 attractions in Singapore for tourists",
    },
]

In [54]:
response = gpt.invoke(messages)

print(response.content)

{
    "attractions": [
        {
            "name": "Marina Bay Sands",
            "address": "10 Bayfront Ave, Singapore 018956",
            "category": "Hotel",
            "recommended_visit_duration": "120",
            "description": "Marina Bay Sands is an integrated resort fronting Marina Bay in Singapore. At its opening in 2010, it was billed as the world's most expensive standalone casino property at S$8 billion, including the land cost."
        },
        {
            "name": "Gardens by the Bay",
            "address": "18 Marina Gardens Dr, Singapore 018953",
            "category": "Park",
            "recommended_visit_duration": "180",
            "description": "Gardens by the Bay is a nature park spanning 101 hectares of reclaimed land in central Singapore, adjacent to the Marina Reservoir. The park consists of three waterfront gardens: Bay South Garden, Bay East Garden, and Bay Central Garden."
        },
        {
            "name": "Sentosa Island",
          

We want 100 places, but the LLM only give us partial result

how to solve this problem?

1. ask fewer each time, ask for 20 items, and repeated with context
1. generate 100 names first and ask for the descriptions and so on for each of them

![Attractions Workflow](https://i.imgur.com/fuyiGCs.png)



In [67]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain.prompts import ChatPromptTemplate


class State(TypedDict):
    # input is like "100, Singapore"
    input: str
    num: int
    tasks: list
    places: list
    current_places: list
    output: str

In [69]:
from pydantic import BaseModel, Field


class Attraction(BaseModel):
    name: str = Field(description="Name of the attraction")
    address: str = Field(description="Address of the attraction")
    category: str = Field(description="Category of the attraction")
    recommended_visit_duration: int = Field(
        description="Recommended visit duration in minutes"
    )
    description: str = Field(description="Description of the attraction")


class Attractions(BaseModel):
    places: list[Attraction] = Field(description="List of attractions")

In [70]:
def generate_task(state: State):
    """break the task into smaller tasks"""

    num, city = state["input"].split(",")
    num = int(num)

    tasks = (
        (
            [f"Generate a list of 20 attractions in {city} for tourists"]
            + [f"Generate 20 more attractions in {city} for tourists"]
            * ((num - 20) // 20)
            + (
                [f"Generate {num % 20} more attractions in {city} for tourists"]
                if num % 20
                else []
            )
        )
        if num >= 20
        else [f"Generate a list of {num} attractions in {city} for tourists"]
    )
    return {"tasks": tasks}

In [71]:
generate_task(State(input="19,Singapore"))

{'tasks': ['Generate a list of 19 attractions in Singapore for tourists']}

In [72]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a professional travel planner. Please generate the attractions based on the task and do not repeat from the already known places.",
        ),
        (
            "human",
            """\
                Here is the task:
                {task}

                Here are the places I have already known, please don't repeat them:
                {places}
                """,
        ),
    ]
)

executor = prompt | gpt.with_structured_output(Attractions)

In [75]:
results = executor.invoke(
    {
        "task": "Generate a list of 19 attractions in Singapore for tourists",
        "places": [],
    }
)

In [76]:
print(results)

places=[Attraction(name='Haw Par Villa', address='262 Pasir Panjang Rd, Singapore 118628', category='Cultural Theme Park', recommended_visit_duration=60, description='Haw Par Villa is a unique theme park with over 1,000 statues and 150 dioramas depicting Chinese mythology, folklore, legends, history, and illustrations of various aspects of Confucianism.'), Attraction(name='The Southern Ridges', address='Mount Faber Park, Telok Blangah Hill Park, HortPark, Kent Ridge Park & Labrador Nature Reserve, Singapore', category='Nature Trail', recommended_visit_duration=180, description='The Southern Ridges is a 10-kilometer trail that connects several parks along the southern ridge of Singapore, offering panoramic views and a rich variety of flora and fauna.'), Attraction(name='Gillman Barracks', address='9 Lock Rd, Singapore 108937', category='Contemporary Art', recommended_visit_duration=90, description='A contemporary art cluster in a former military barracks, featuring leading international

In [77]:
def execute(state: State):
    """call the LLM to perform the task"""
    # 1 get one task from the tasks, the pop it
    # 2 call the LLM to perform the task and get the result
    task, plases = state["tasks"][0], state["places"]
    state["tasks"].pop(0)
    res = executor.invoke({"task": task, "places": plases})

    return {"current_places": res.places}

In [78]:
dedup_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant in deduplicating the attractions. Please remove the duplicates from the current places and add them to the known places.",
        ),
        (
            "human",
            """\
                Here are the places already known:
                {places}

                Here are the newly generated places:
                {current_places}

                Return the current places without duplicates.
                """,
        ),
    ]
)

deduplicator = dedup_prompt | gpt.with_structured_output(Attractions)

In [79]:
def dedup(state: State):
    """"""
    # call the LLM to deduplicate the places
    res = deduplicator.invoke(
        {"current_places": state["current_places"], "places": state["plases"]}
    )
    places = res.places + state["plases"]
    return {"places": places}

In [80]:
def should_end(state: State):
    if len(state["places"]) >= state["num"]:
        return "hit target"
    return "below target"

In [81]:
builder = StateGraph(State)

builder.set_entry_point("generator")
builder.add_node("generator", generate_task)
builder.add_node("executor", execute)
builder.add_node("deduplicator", dedup)

builder.add_edge("generator", "executor")
builder.add_edge("executor", "deduplicator")
builder.add_conditional_edges(
    "deduplicator", should_end, {"below target": "executor", "hit target": END}
)

graph = builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
res = graph.invoke({"input": "Singapore,100"})
res["places"]

### Langserve! If time permit!

**LangServe** is a tool within LangChain that makes it easy to deploy your language model applications in production. It allows you to serve your LangChain workflows as APIs, which is super helpful when you want to scale or integrate your application into larger systems.

---

### Why is it helpful?
- **Easy Deployment**: LangServe simplifies the process of turning your LangChain workflows into APIs.
- **Scalability**: It helps scale your application so that multiple users or systems can interact with it.
- **Seamless Integration**: LangServe allows you to connect your application with other services through an API, making it accessible in different environments.

---

### When should you use it?
- When you need to **deploy** your LangChain project for **real-world use**.
- If you want to **share your model** as a service for others to use (e.g., for a web app or another application).
- When you need to **scale** your language model applications to handle more users or requests.

In short, LangServe is perfect for moving your LangChain project from development to production!


In [55]:
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes
import nest_asyncio
import uvicorn

In [56]:
# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

In [57]:
# 2. Create chain
chain = prompt_template | gpt | StrOutputParser()

In [58]:
# 3. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

In [59]:
# Allow asyncio to work within a running event loop (e.g., in Jupyter)
nest_asyncio.apply()

In [60]:
add_routes(
    app,
    chain,
    path="/finalchain3",
)

In [61]:
from fastapi.testclient import TestClient
# Initialize TestClient
client = TestClient(app)

# Test simple route
response = client.post("/finalchain3", json={"language": "Japanese"})

In [62]:
print(response)

<Response [404 Not Found]>


In [63]:
# 4. Adding chain route
add_routes(
    app,
    chain,
    path="/finalchain3",
)

if __name__ == "__main__":

  uvicorn.run(app, host="0.0.0.0", port=8000)

ValueError: A runnable already exists at path: /finalchain3. If adding multiple runnables make sure they have different paths.

In [ ]:
app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple api server using Langchain's Runnable interfaces",
)

add_routes(
    app,
    ChatOpenAI(model="gpt-3.5-turbo-0125"),
    path="/openai",
)

model = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

add_routes(
    app,
    prompt | model,
    path="/joke",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host='localhost', port=8000)

In [ ]:
prompt=ChatPromptTemplate.from_template("provide me an essay about {topic}")
prompt1=ChatPromptTemplate.from_template("provide me a poem about {topic}")

add_routes(
    app,
    prompt|model,
    path="/essay"

)

add_routes(
    app,
    prompt1|model,
    path="/poem"

)

if __name__=="__main__":
    uvicorn.run(app,host="localhost",port=8000)